In [17]:
%load_ext autoreload
%autoreload 2
from ProfileBuilder import Graphlet

import matplotlib.pyplot as plt
from sklearn import svm, datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
graphlet = Graphlet('data/annotated-trace.csv')

In [43]:
X = graphlet.activity_graphlets
y = graphlet.get_graphlets_label()


In [ ]:
#custom_kernel
clf = svm.SVC(kernel=custom_kernel)